# Исследование опроса клиентов телекомунникацонной компании

## Выгрузка данных

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
           SELECT u.user_id,
                u.lt_day,
                CASE 
                    WHEN lt_day <=365 THEN 'новый'
                    ELSE 'старый'
                END AS is_new,
                u.age,
                CASE
                    WHEN gender_segment = 0 THEN 'мужчина'
                    WHEN gender_segment = 1 THEN 'женщина'
                    ELSE 'неизвестно'
                END AS gender_segment,
                u.os_name,
                u.cpe_type_name,
                l.country,
                l.city,
                SUBSTRING(age.title, 4, 5) AS age_segment,
                ts.title AS traffic_segment,
                ls.title As lifetime_segment,
                u.nps_score,
                CASE
                    WHEN nps_score >= 9 THEN 'cторонники'
                    WHEN nps_score >= 7 AND nps_score <=8 THEN 'нейтралы'
                    ELSE 'критики'
                    END AS nps_group
           FROM user As u
           JOIN location AS l ON u.location_id = l.location_id
           JOIN age_segment AS age ON u.age_gr_id = age.age_gr_id
           JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
           JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
           WHERE u.lt_day > 0;
        """

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,04 1-5,08 36+,10,cторонники
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,04 1-5,08 36+,10,cторонники
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,08 20-25,06 13-24,10,cторонники


Сохраняем данные в csv формате

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

[Ссылка на дашборд](https://public.tableau.com/views/NPS_16762041189940/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link)

[Ссылка на презентацию](https://drive.google.com/file/d/1tcmPJ-6c5Vuf3bh_IE6IDfV5xcva4VYZ/view?usp=sharing)